In [3]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [4]:
#Load and Preprocess the Dataset
dataset_path = "/kaggle/input/mrl-dataset/train/"
categories = ["Closed_Eyes", "Open_Eyes"]

data = []
labels = []

img_size = 64  # Resize all images to 64x64

for category in categories:
    path = os.path.join(dataset_path, category)
    class_num = categories.index(category)  # 0 for Closed_Eyes, 1 for Open_Eyes

    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)  # Read in grayscale
            img_array = cv2.resize(img_array, (img_size, img_size))  # Resize
            data.append(img_array)
            labels.append(class_num)
        except Exception as e:
            print("Error loading image:", e)

# Convert to NumPy arrays
data = np.array(data).reshape(-1, img_size, img_size, 1) / 255.0  # Normalize
labels = np.array(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)


# A1

In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 682,753 (2.60 MB)

 Trainable params: 682,753 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


# A2

In [ ]:
# Feature extractor model (outputs from the penultimate layer)
feature_extractor = keras.Model(inputs=model.input, outputs=model.layers[-2].output)

# Extract features
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

print("Feature shape:", X_train_features.shape)  # (num_samples, feature_vector_size)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_features, y_train)

y_pred_knn = knn.predict(X_test_features)
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_features, y_train)

y_pred_rf = rf.predict(X_test_features)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")


In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear')
svm.fit(X_train_features, y_train)

y_pred_svm = svm.predict(X_test_features)
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_features, y_train)

y_pred_nb = nb.predict(X_test_features)
print(f"Naïve Bayes Accuracy: {accuracy_score(y_test, y_pred_nb):.4f}")


# **Regression Models**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train_features, y_train)

y_pred_lr = lr.predict(X_test_features)
print(f"Linear Regression MSE: {mean_squared_error(y_test, y_pred_lr):.4f}")


In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(X_train_features, y_train)

y_pred_lasso = lasso.predict(X_test_features)
print(f"LASSO Regression MSE: {mean_squared_error(y_test, y_pred_lasso):.4f}")


In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf')
svr.fit(X_train_features, y_train)

y_pred_svr = svr.predict(X_test_features)
print(f"SVR MSE: {mean_squared_error(y_test, y_pred_svr):.4f}")


# A4 *Metrics for Generative Models*

**Frechet Inception Distance (FID)**

* Measures similarity between real and generated images using feature statistics.
* Lower FID means better generation quality.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from scipy.linalg import sqrtm

def calculate_fid(real_images, generated_images):
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
    
    real_features = model.predict(real_images)
    gen_features = model.predict(generated_images)
    
    mu1, sigma1 = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = np.mean(gen_features, axis=0), np.cov(gen_features, rowvar=False)
    
    fid = np.linalg.norm(mu1 - mu2) + np.trace(sigma1 + sigma2 - 2 * sqrtm(sigma1.dot(sigma2)))
    return fid

# Example Usage
# fid_score = calculate_fid(real_images, generated_images)
# print(f"FID Score: {fid_score:.4f}")


**Structural Similarity Index (SSIM)**
* Measures how structurally similar two images are (range: -1 to 1, where 1 is identical).

In [ ]:
# Structural Similarity Index (SSIM)
from skimage.metrics import structural_similarity as ssim

def compute_ssim(img1, img2):
    return ssim(img1, img2, multichannel=True)

# Example usage:
# ssim_score = compute_ssim(real_image, generated_image)
# print(f"SSIM Score: {ssim_score:.4f}")


**Inception Score (IS)**
* Evaluates image quality based on Inception network predictions (higher IS means better diversity & quality).

In [3]:
# Inception Score (IS)
from scipy.stats import entropy

def inception_score(p_yx, eps=1e-16):
    p_y = np.expand_dims(p_yx.mean(axis=0), 0)
    kl_div = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
    return np.exp(kl_div.sum(axis=1).mean())

# Example usage:
# is_score = inception_score(model_predictions)
# print(f"Inception Score: {is_score:.4f}")


In [ ]:
def main():
    # Generate Random Fake Data for Testing
    real_images = np.random.rand(10, 299, 299, 3)  # 10 real images
    generated_images = np.random.rand(10, 299, 299, 3)  # 10 generated images
    img1 = np.random.rand(299, 299, 3)  # Single random image
    img2 = np.random.rand(299, 299, 3)  # Another random image

    # Simulate Fake Model Predictions for Inception Score
    fake_predictions = np.random.rand(10, 5)  # 10 images, 5 class probabilities
    fake_predictions = fake_predictions / fake_predictions.sum(axis=1, keepdims=True)  # Normalize to sum=1

    # Compute Scores
    fid_score = calculate_fid(real_images, generated_images)
    ssim_score = compute_ssim(img1, img2)
    is_score = inception_score(fake_predictions)

    # Print Scores
    print(f"FID Score: {fid_score:.4f}")
    print(f"SSIM Score: {ssim_score:.4f}")
    print(f"Inception Score: {is_score:.4f}")

# Run the main function
if __name__ == "__main__":
    main()
